In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import re

In [67]:
def scrolla_pra_baixo(driver: webdriver):
    tempo_pausa_scroll = 0.5
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(tempo_pausa_scroll)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [3]:
def obtem_codigo_fonte(url: str) -> str:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path = '../chromedriver.exe', chrome_options = chrome_options)
    driver.get(url)
    scrolla_pra_baixo(driver)

    return driver.page_source

In [4]:
url_nike = 'https://www.nike.com.br/snkrs#estoque'
bs_estoque = BeautifulSoup(obtem_codigo_fonte(url_nike), 'html.parser')

lista_de_urls_snkrs = list()
for i, url_snkrs in enumerate(bs_estoque.find_all('div', {'class' : 'produto produto--comprar'})):
    href = url_snkrs.find('a', href = True)['href']
    lista_de_urls_snkrs.append(href)
    
lista_de_urls_snkrs[:5]

<ipython-input-3-7172526d085b>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = '../chromedriver.exe', chrome_options = chrome_options)


['https://www.nike.com.br/air-jordan-1-low-og-153-169-211-315618',
 'https://www.nike.com.br/air-jordan-7-153-169-211-315592',
 'https://www.nike.com.br/womens-lahar-low-canvas-1-16-210-330586',
 'https://www.nike.com.br/womens-lahar-low-canvas-1-16-210-330564',
 'https://www.nike.com.br/air-max-90-153-169-211-330199']

In [89]:
lista_de_snkrs = list()
for i, url in enumerate(lista_de_urls_snkrs):
    try:
        dicionario = dict()
        bs_snkrs = BeautifulSoup(obtem_codigo_fonte(url), 'html.parser')
        dicionario['Valor'] = bs_snkrs.find('span', {'class' : '--valores'}).text
        dicionario['Nome'] = bs_snkrs.find('div', {'class' : 'nome-preco-produto'}).find('span').text


        padrao_regex = r'data-tamanho="(.|..|...|....)"'
        dicionario['Tamanhos'] = [re.findall(padrao_regex, str(tamanho)) 
                                  for tamanho in bs_snkrs.find_all('input', 
                                                              {'class' : 'tamanho__item_pdp js-tamanho__item_pdp'})]
        dicionario['Link'] = url
        
    except NoSuchElementException as e:
        print(f'Elemento não encontrado na url: {i} - {url}')
        continue
        
    finally:
        lista_de_snkrs.append(dicionario)

<ipython-input-3-7172526d085b>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = '../chromedriver.exe', chrome_options = chrome_options)


In [91]:
data = pd.DataFrame(lista_de_snkrs, columns = ['Nome', 'Valor', 'Tamanhos', 'Link']).set_index('Nome')
data.head()

,Valor,Tamanhos,Link
Nome,,,
Air Jordan 1 Low OG,"R$ 899,99","[[45], [46]]",https://www.nike.com.br/air-jordan-1-low-og-15...
Air Jordan 7,"R$ 1.299,99","[[39], [40], [41]]",https://www.nike.com.br/air-jordan-7-153-169-2...
Women's Lahar Low Canvas,"R$ 649,99","[[33,5], [34], [35,5], [36], [37,5], [38], [39...",https://www.nike.com.br/womens-lahar-low-canva...
Women's Lahar Low Canvas,"R$ 649,99","[[33,5], [34], [35,5], [36], [37,5], [38]]",https://www.nike.com.br/womens-lahar-low-canva...
Air Max 90,"R$ 749,99","[[36], [37], [38], [39], [39,5], [40,5], [41],...",https://www.nike.com.br/air-max-90-153-169-211...


In [96]:
dados_nulos = data.isna().sum()
print(dados_nulos)

Valor       0
Tamanhos    0
Link        0
dtype: int64


In [97]:
_ = data.dropna(inplace = True)

In [109]:
data.to_excel('Estoque_Nike_Snkrs.xls', encoding = 'UTF-8')